In [48]:
from selenium import webdriver
import time
import pandas as pd
import random
from dotenv import load_dotenv
import os

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [49]:
url1 = 'https://www.linkedin.com/jobs/search/?keywords=Software%20Engineer%20Intern&location=United%20States&geoId=103644278&f_TPR=r604800&position=1&pageNum=0'

In [52]:
load_dotenv()
service = Service(os.getenv("CHROMEDRIVER_PATH"))
driver = webdriver.Chrome(service=service)
driver.get(url1)
driver.implicitly_wait(10)

In [53]:
actions = ActionChains(driver)
actions.move_by_offset(random.randint(1, 5), random.randint(1, 5)).click().perform()

In [19]:
n = driver.find_element(By.CLASS_NAME, "results-context-header__job-count").text

In [20]:
n

'354'

In [21]:
n = pd.to_numeric(n)
n

354

In [ ]:
i = 1
numScroll = (n/25)+1

while(i <= numScroll):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    
    try:
        see_more_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='See more jobs']"))
        )
        see_more_button.click()
        time.sleep(random.randint(3, 6))
    except:
        pass
        time.sleep(random.randint(2, 5))

In [23]:
companyName = []

In [ ]:
numJobs = 0

for j in range(n):
    try:
        company = driver.find_elements(By.CLASS_NAME, "base-search-card__subtitle")[j].text
        numJobs = numJobs+1
        companyName.append(company)
    
    except IndexError:
        pass

In [25]:
len(companyName)

193

In [26]:
titleName = []

In [27]:
for i in range(numJobs):
    try:
        title = driver.find_elements(By.CLASS_NAME, "base-search-card__title")[i].text
        titleName.append(title)
    
    except IndexError:
        pass

In [28]:
len(titleName)

193

In [29]:
linkList = []

In [30]:
for k in range(n):
    try:
        link_element = driver.find_elements(By.CLASS_NAME, "base-card__full-link")[k].get_attribute('href')
        if link_element is not None:
            linkList.append(link_element)
    except IndexError:
        pass

In [31]:
linkList

['https://www.linkedin.com/jobs/view/software-engineering-intern-nrec-at-carnegie-mellon-university-4144970349?position=1&pageNum=0&refId=WNqG6qB1QMjp%2F95GmXLvSA%3D%3D&trackingId=zdrK9ws9l3h9bcvQx8g%2Fkg%3D%3D',
 'https://www.linkedin.com/jobs/view/computational-science-software-engineering-intern-summer-2025-at-lam-research-4058647382?position=2&pageNum=0&refId=WNqG6qB1QMjp%2F95GmXLvSA%3D%3D&trackingId=onF9oFQ51YzAISyu5wMXew%3D%3D',
 'https://www.linkedin.com/jobs/view/software-engineering-business-analyst-intern-at-ping-4143542919?position=3&pageNum=0&refId=WNqG6qB1QMjp%2F95GmXLvSA%3D%3D&trackingId=ZplIxXv9Vt5SXbS%2Bx3S%2FAw%3D%3D',
 'https://www.linkedin.com/jobs/view/strategic-student-program-software-development-intern-eda-summer-2025-at-siemens-eda-siemens-digital-industries-software-4142506026?position=4&pageNum=0&refId=WNqG6qB1QMjp%2F95GmXLvSA%3D%3D&trackingId=pt%2B7AKitpe7pS6yKN7UQCg%3D%3D',
 'https://www.linkedin.com/jobs/view/2025-summer-intern-system-engineer-sequencing-de

In [32]:
remainingLinks = []
indexList = []

for k in range(n):
    try:
        link_element = driver.find_elements(By.CLASS_NAME, "base-card--link")[k].get_attribute('href')
        if link_element is not None:
            indexList.append(k)
            remainingLinks.append(link_element)
    except IndexError:
        pass

In [33]:
for i in indexList:
    linkList.insert(i, remainingLinks[0])
    remainingLinks.pop(0)

In [34]:
indexList

[]

In [35]:
datePosted = []

In [36]:
for m in range(n):
    try:
        time_element = driver.find_elements(By.CSS_SELECTOR, ".job-search-card__listdate, .job-search-card__listdate--new")[m].get_attribute('datetime')
        datePosted.append(time_element)
    except IndexError:
        pass

In [37]:
len(datePosted)

193

In [38]:
companyData = pd.DataFrame(companyName, columns=["Company"])
titleData = pd.DataFrame(titleName, columns=["Title"])
linkData = pd.DataFrame(linkList, columns=["Links"])
dateData = pd.DataFrame(datePosted, columns=["Date Posted"])

In [39]:
companyData

,Company
0,Carnegie Mellon University
1,Lam Research
2,PING
3,Siemens EDA (Siemens Digital Industries Software)
4,Roche
...,...
188,Peraton
189,Terracon
190,Peraton
191,Terracon


In [40]:
titleData

,Title
0,Software Engineering Intern - NREC
1,Computational Science Software Engineering Int...
2,Software Engineering/Business Analyst Intern
3,Strategic Student Program: Software Developmen...
4,2025 Summer Intern - System Engineer/Sequencin...
...,...
188,Senior Software Engineer
189,Geotechnical Field Engineer
190,Senior-Level GEOINT Systems Engineer
191,Field Engineer/Building Enclosure Consultant


In [41]:
linkData

,Links
0,https://www.linkedin.com/jobs/view/software-en...
1,https://www.linkedin.com/jobs/view/computation...
2,https://www.linkedin.com/jobs/view/software-en...
3,https://www.linkedin.com/jobs/view/strategic-s...
4,https://www.linkedin.com/jobs/view/2025-summer...
...,...
188,https://www.linkedin.com/jobs/view/senior-soft...
189,https://www.linkedin.com/jobs/view/geotechnica...
190,https://www.linkedin.com/jobs/view/senior-leve...
191,https://www.linkedin.com/jobs/view/field-engin...


In [42]:
jobData = pd.DataFrame()
jobData = pd.concat([companyData, titleData, dateData, linkData], axis=1)
numColumns = (jobData.shape[1]*2)-1

for i in range(1, numColumns, 2):
    jobData.insert(i, f'Empty_{i}', '')
    
jobData.index = jobData.index+1

In [ ]:
jobData

,Company,Empty_1,Title,Empty_3,Date Posted,Empty_5,Links
1,Carnegie Mellon University,,Software Engineering Intern - NREC,,2025-02-05,,https://www.linkedin.com/jobs/view/software-en...
2,Lam Research,,Computational Science Software Engineering Int...,,2025-02-05,,https://www.linkedin.com/jobs/view/computation...
3,PING,,Software Engineering/Business Analyst Intern,,2025-02-06,,https://www.linkedin.com/jobs/view/software-en...
4,Siemens EDA (Siemens Digital Industries Software),,Strategic Student Program: Software Developmen...,,2025-02-05,,https://www.linkedin.com/jobs/view/strategic-s...
5,Roche,,2025 Summer Intern - System Engineer/Sequencin...,,2025-02-05,,https://www.linkedin.com/jobs/view/2025-summer...
...,...,...,...,...,...,...,...
189,Peraton,,Senior Software Engineer,,2025-02-07,,https://www.linkedin.com/jobs/view/senior-soft...
190,Terracon,,Geotechnical Field Engineer,,2025-02-07,,https://www.linkedin.com/jobs/view/geotechnica...
191,Peraton,,Senior-Level GEOINT Systems Engineer,,2025-02-09,,https://www.linkedin.com/jobs/view/senior-leve...
192,Terracon,,Field Engineer/Building Enclosure Consultant,,2025-02-05,,https://www.linkedin.com/jobs/view/field-engin...


In [44]:
jobData.to_csv("LinkedIn_SWE_Intern_Jobs.csv")

In [45]:
driver.quit()